In [1]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
df = pd.read_excel (r"D:\work\Tasks\Data_13062022\955\FinalData_Train_Test\FinalDataArrange.xlsx",0, header=0)
df.shape

(955, 2010)

In [2]:
data = df.values
data.shape

(955, 2010)

In [3]:
X, y = data[:,1:2009], data[:, -1]
print(X.shape, y.shape)

(955, 2008) (955,)


In [4]:
def KS(X,y,train_size):
    
    #Normalize the features using MinMaxScaler
   # scaler = MinMaxScaler()
    #X_norm = scaler.fit_transform(X)
    
    #Number of points in the training set
    K = round(X.shape[0]*train_size)
    if(K<2):
        raise Exception('Training set size should be at least 2')
        
    #Find two most separated points
    pairwise_distance = cdist(X,X,metric='jaccard')
    most_separated_points = np.unravel_index(np.argmax(pairwise_distance), pairwise_distance.shape)
    trainingset_indices = [most_separated_points[0], most_separated_points[1]]
    trainingset_size = len(trainingset_indices)
    while(trainingset_size < K):
        selection_indices = [i for i in range(X.shape[0]) if i not in trainingset_indices]
        selection_distances = pairwise_distance[np.ix_(selection_indices, trainingset_indices)]
        separation_distances = selection_distances.min(axis=1)
        trainingset_indices.append(selection_indices[np.argmax(separation_distances)])
        trainingset_size = len(trainingset_indices)
        
    testset_indices = [i for i in range(X.shape[0]) if i not in trainingset_indices]
    
    return(trainingset_indices, testset_indices)

In [3]:
def SPXY(X,y,train_size):
    #Normalize the features using MinMaxScaler
    #scaler_X = MinMaxScaler()
    #scaler_y = MinMaxScaler()
    #X_norm = scaler_X.fit_transform(X)
    #Normalize the responses using MinMaxScaler
    #Reshape y if y is 1-dimensional
    if(y.ndim == 1):
        y=y.reshape(-1,1)
    #y_norm = scaler_y.fit_transform(y)
    
    #Number of points in training set
    K = round(X.shape[0]*train_size)
    if(K<2):
        raise Exception('Training set size should be at least 2')
    #Find two most separated points
    pairwise_distance_X = cdist(X,X,metric='jaccard')
    pairwise_distance_X = pairwise_distance_X/pairwise_distance_X.max()
    pairwise_distance_y = cdist(y,y,metric='jaccard')
    pairwise_distance_y = pairwise_distance_y/pairwise_distance_y.max()
    pairwise_distance = pairwise_distance_X + pairwise_distance_y
    most_separated_points = np.unravel_index(np.argmax(pairwise_distance), pairwise_distance.shape)
    trainingset_indices = [most_separated_points[0], most_separated_points[1]]
    trainingset_size = len(trainingset_indices)
    while(trainingset_size < K):
        selection_indices = [i for i in range(X.shape[0]) if i not in trainingset_indices]
        selection_distances = pairwise_distance[np.ix_(selection_indices, trainingset_indices)]
        separation_distances = selection_distances.min(axis=1)
        trainingset_indices.append(selection_indices[np.argmax(separation_distances)])
        trainingset_size = len(trainingset_indices)
        
    testset_indices = [i for i in range(X.shape[0]) if i not in trainingset_indices]
    
    return(trainingset_indices, testset_indices)

In [5]:
trainingset_indices,testset_indices = KS(X,y,0.75)

In [7]:
len(testset_indices)

239

In [8]:
X_train_df=df.iloc[trainingset_indices]
X_test_df=df.iloc[testset_indices]

In [ ]:
X_train_df.to_excel("D:/work/Tasks/Data_13062022/955/FinalData_Train_Test/KS/trainSet_KS.xlsx",sheet_name='train')
X_test_df.to_excel("D:/work/Tasks/Data_13062022/955/FinalData_Train_Test/KS/testSet_KS.xlsx",sheet_name='test')